In [0]:
import pyspark.sql.functions as F
from datetime import datetime, timedelta
from pyspark.sql import SparkSession

from funcoes.gerais import *

In [0]:
tabela_vendas_silver = "estudo.default.tvendas_silver"
tabela_cluster_silver = "estudo.default.tcluster_silver"
tabela_produto_cluster_gold = "estudo.default.tproduto_cluster_gold"

In [0]:
df_vendas = spark.table(tabela_vendas_silver)

In [0]:
df_vendas = df_vendas.withColumn("media_tickets_por_compra", F.round(F.col("total_tickets_compras") / F.col("total_compras"), 2)) \
    .withColumn("porcentagem_viagens_com_volta", F.round(F.col("total_compras_com_volta") / F.col("total_compras") * 100, 2)) \
        .select(
            "fk_contact",
            "total_compras",
            "valor_total_compra",
            "valor_medio_compra",
            "total_tickets_compras",
            "media_tickets_por_compra",
            "total_compras_com_volta",
            "porcentagem_viagens_com_volta",
            "local_partida_ida_local_destino_ida_frequente",
            "local_partida_volta_local_destino_volta_frequente",
            "dia_da_semana_preferido_de_compra",
            "epoca_do_mes_preferido_da_compra",
            "periodo_do_dia_preferido_para_compra",
            "epoca_do_ano_preferido_da_compra"
        ).distinct()

In [0]:
df_cluster = spark.table(tabela_cluster_silver)

In [0]:
df_join = df_vendas \
    .join(df_cluster, on=["fk_contact"], how="left")

In [0]:
df_final = df_join.select(
        "fk_contact",
        F.col("cluster").alias("cd_cluster"),
        F.col("segmento_do_cliente").alias("ds_cluster"),
        "recencia_dias",
        F.round("valor_total_gasto", 2).alias("valor_total_gasto"),
        F.round("valor_medio_compra", 2).alias("valor_medio_compra"),
        F.col("frequencia_de_compras").alias("quantidade_compras"),
        "total_tickets_compras",
        "media_tickets_por_compra",
        "total_compras_com_volta",
        "porcentagem_viagens_com_volta",
        F.col("local_partida_ida_local_destino_ida_frequente").alias("rota_ida_frequente"),
        F.col("local_partida_volta_local_destino_volta_frequente").alias("rota_volta_frequente"),
        F.col("dia_da_semana_preferido_de_compra").alias("dia_semana_preferido_compra"),
        F.col("epoca_do_mes_preferido_da_compra").alias("dia_mes_preferido_compra"),
        F.col("periodo_do_dia_preferido_para_compra").alias("periodo_dia_preferido_compra"),
        F.col("epoca_do_ano_preferido_da_compra").alias("mes_preferido_compra")
    )

In [0]:
df_final.count()

In [0]:
# Escrita na tabela que será usada nos estudos e desenvolvimentos
escrita_tabela(df_final, tabela_produto_cluster_gold)